In [52]:
import pandas as pd
import numpy as np

df1 = pd.read_excel("/content/Input.xlsx")

In [ ]:
urls = [url for url in df1['URL']]

In [73]:
import requests
from bs4 import BeautifulSoup

def scrape_article_text(url):
    response = requests.get(url)

    if response.status_code == 200:

        soup = BeautifulSoup(response.text, 'html.parser')


        article_text_element = soup.find('div',class_='td-post-content')

        if article_text_element:
            pre_tag = article_text_element.find('pre')
            if pre_tag:
                pre_tag.decompose()

            text = article_text_element.get_text()
            return text
        else:
            print(f'No element with class containing "td-post-content" found in {url}')
    else:
        print(f'Failed to retrieve data from {url}.URL status code: {response.status_code}')

In [75]:
article_text_list = []

for url in urls:
    article_text = scrape_article_text(url)
    if article_text:
        article_text_list.append(article_text)
    else:
        article_text_list.append(' ')

Failed to retrieve data from https://insights.blackcoffer.com/how-neural-networks-can-be-applied-in-various-areas-in-the-future//n URL status code: 404
Failed to retrieve data from https://insights.blackcoffer.com/covid-19-environmental-impact-for-the-future//n URL status code: 404


In [ ]:
for i in range(len(article_text_list)):
  article_text_list[i]= article_text_list[i].replace('\n','')

article_text_list

In [ ]:
stopwords=[]
stopwords_merged_file = '/content/Merged.txt'

with open(stopwords_merged_file, 'r') as file:
    for line in file:
        word = line.strip()
        word=word.lower()
        stopwords.append(word)

In [ ]:
cleaned_article_text = [' ']*len(article_text_list)

for i in range(len(article_text_list)):
  for w in stopwords:
    cleaned_article_text[i]= article_text_list[i].replace(' '+w+' ',' ')

In [ ]:
cleaned_article_text[37]

' '

In [ ]:
positive_words=[]
positive_file = '/content/positive-words.txt'

with open(positive_file,'r') as file:
  for line in file:
    word = line.strip()
    word = word.lower()
    positive_words.append(word)

In [ ]:
positive_words

In [ ]:
positive_score = [0]*len(article_text_list)
for i in range(len(article_text_list)):
  for word in positive_words:
    for letter in cleaned_article_text[i].lower().split(' '):
      if letter==word:
        positive_score[i]+=1


In [ ]:
positive_score

In [ ]:
negative_words=[]
negative_file = '/content/negative-words.txt'

with open(negative_file,'r',encoding = "ISO-8859-1") as file:
  for line in file:
    word = line.strip()
    word = word.lower()
    negative_words.append(word)

In [ ]:
negative_score = [0]*len(article_text_list)
for i in range(len(article_text_list)):
  for word in negative_words:
    for letter in cleaned_article_text[i].lower().split(' '):
      if letter==word:
        negative_score[i]+=1


In [ ]:
negative_score

In [21]:
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [23]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [26]:
sentences = []
words=[]
for article in cleaned_article_text:
  sentences.append(len(sent_tokenize(article)))
  words.append(len(word_tokenize(article)))

In [31]:
words = np.array(words)
sentences = np.array(sentences)

In [77]:
complex_words = []
sylabble_counts = []

for article in cleaned_article_text:
  sylabble_count=0
  d=article.split()
  ans=0
  for word in d:
    count=0
    for i in range(len(word)):
      if(word[i]=='a' or word[i]=='e' or word[i] =='i' or word[i] == 'o' or word[i] == 'u'):
           count+=1
          #  print(words[i])
      if(i==len(word)-2 and (word[i]=='e' and word[i+1]=='d')):
        count-=1;
      if(i==len(word)-2 and (word[i]=='e' and word[i]=='s')):
        count-=1;
    sylabble_count+=count
    if(count>2):
        ans+=1
  sylabble_counts.append(sylabble_count)
  complex_words.append(ans)

In [33]:
total_characters = []
for article in cleaned_article_text:
  characters = 0
  for word in article.split():
    characters+=len(word)
  total_characters.append(characters)

In [34]:
personal_nouns = []
personal_noun =['I', 'we','my', 'ours','and' 'us','My','We','Ours','Us','And']
for article in cleaned_article_text:
  ans=0
  for word in article:
    if word in personal_noun:
      ans+=1
  personal_nouns.append(ans)

In [53]:
df = df1

In [ ]:
df['POSITIVE SCORE'] = positive_score
df

In [56]:
df['NEGATIVE SCORE'] = negative_score

In [57]:
df['POLARITY SCORE'] = (df['POSITIVE SCORE']-df['NEGATIVE SCORE'])/ ((df['POSITIVE SCORE'] +df['NEGATIVE SCORE']) + 0.000001)

In [58]:
df['SUBJECTIVITY SCORE'] = (df['POSITIVE SCORE'] + df['NEGATIVE SCORE'])/( (words) + 0.000001)

In [59]:
df['AVG SENTENCE LENGTH'] = np.array(words)/np.array(sentences)

<ipython-input-59-b4102d65ebc0>:1: RuntimeWarning: invalid value encountered in divide
  df['AVG SENTENCE LENGTH'] = np.array(words)/np.array(sentences)


In [60]:
df['PERCENTAGE OF COMPLEX WORDS'] = np.array(complex_words)/np.array(words)

<ipython-input-60-ad31425619b8>:1: RuntimeWarning: invalid value encountered in divide
  df['PERCENTAGE OF COMPLEX WORDS'] = np.array(complex_words)/np.array(words)


In [61]:
df['FOG INDEX'] = 0.4 * (df['AVG SENTENCE LENGTH'] + df['PERCENTAGE OF COMPLEX WORDS'])

In [62]:
df['AVG NUMBER OF WORDS PER SENTENCES'] = df['AVG SENTENCE LENGTH']

In [63]:
df['COMPLEX WORD COUNT'] = complex_words

In [64]:
df['WORD COUNT'] = words

In [65]:
df['SYLLABLE PER WORD'] = np.array(sylabble_counts)/np.array(words)

<ipython-input-65-ede53e0923f2>:1: RuntimeWarning: invalid value encountered in divide
  df['SYLLABLE PER WORD'] = np.array(sylabble_counts)/np.array(words)


In [66]:
df['PERSONAL PRONOUN'] = personal_nouns

In [67]:
df['AVG WORD LENGTH'] = np.array(total_characters)/np.array(words)

<ipython-input-67-570ce97e01e7>:1: RuntimeWarning: invalid value encountered in divide
  df['AVG WORD LENGTH'] = np.array(total_characters)/np.array(words)


In [68]:
df

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCES,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUN,AVG WORD LENGTH
0,123.0,https://insights.blackcoffer.com/rise-of-telem...,72,19,0.582418,0.050952,31.892857,0.318589,12.884578,31.892857,569,1786,2.023516,12,5.284434
1,321.0,https://insights.blackcoffer.com/rise-of-e-hea...,40,11,0.568627,0.079937,33.578947,0.321317,13.560106,33.578947,205,638,2.000000,4,5.203762
2,2345.0,https://insights.blackcoffer.com/rise-of-e-hea...,23,20,0.069767,0.036165,18.873016,0.253995,7.650804,18.873016,302,1189,1.732548,21,4.867956
3,4321.0,https://insights.blackcoffer.com/rise-of-telem...,39,24,0.238095,0.047511,28.826087,0.288084,11.645669,28.826087,382,1326,1.874057,19,5.165158
4,432.0,https://insights.blackcoffer.com/rise-of-telem...,39,24,0.238095,0.047511,28.826087,0.288084,11.645669,28.826087,382,1326,1.874057,19,5.165158
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
109,50921.0,https://insights.blackcoffer.com/coronavirus-i...,8,19,-0.407407,0.038190,37.210526,0.247525,14.983220,37.210526,175,707,1.801980,6,5.165488
110,51382.8,https://insights.blackcoffer.com/coronavirus-i...,24,48,-0.333333,0.040404,162.000000,0.219978,64.887991,162.000000,392,1782,1.786756,32,4.967452
111,51844.6,https://insights.blackcoffer.com/what-are-the-...,77,20,0.587629,0.050311,31.096774,0.243776,12.536220,31.096774,470,1928,1.727697,15,4.823651
112,52306.4,https://insights.blackcoffer.com/marketing-dri...,27,16,0.255814,0.028859,26.607143,0.235570,10.737085,26.607143,351,1490,1.738255,16,4.919463


In [70]:
df.to_csv('Output.csv')